In [51]:
pip install langchain langchain-classic

Note: you may need to restart the kernel to use updated packages.


In [50]:
pip show langchain

Name: langchain
Version: 1.2.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Users/adeb/llm_engineering/llms/lib/python3.11/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [54]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationChain

## Chat API : LangChain


In [16]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [19]:
#You don’t need to pass the key explicitly in the api as an arg unless you want to override it
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm_model="gpt-4o-mini"
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x34d433fd0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x34dd17fd0>, root_client=<openai.OpenAI object at 0x30c7c9a10>, root_async_client=<openai.AsyncOpenAI object at 0x34dd17ad0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [20]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [22]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)


In [23]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [24]:
customer_style = """American English \
in a calm and respectful tone
"""

In [30]:
customer_email = """
Meine Bestellung wurde nicht geliefert!
"""

In [31]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [32]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [33]:
print(customer_messages[0])

content='Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nMeine Bestellung wurde nicht geliefert!\n```\n' additional_kwargs={} response_metadata={}


In [35]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [36]:
print(customer_response.content)

My order has not been delivered.


## Memory

In [42]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat,
    memory = memory,
    verbose=True
)

In [43]:
conversation.predict(input="Hi, my name is Anirban. I work at Akamai.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Anirban. I work at Akamai.
AI:

> Finished chain.


"Hello, Anirban! It's great to meet you. Akamai is known for its content delivery network and cloud services, right? What do you do there?"

In [44]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Anirban. I work at Akamai.
AI: Hello, Anirban! It's great to meet you. Akamai is known for its content delivery network and cloud services, right? What do you do there?
Human: What is 1+1?
AI:

> Finished chain.


"1 + 1 equals 2! It's a simple arithmetic problem, but it's always good to double-check the basics. Do you have any other questions or topics you'd like to discuss?"

In [45]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Anirban. I work at Akamai.
AI: Hello, Anirban! It's great to meet you. Akamai is known for its content delivery network and cloud services, right? What do you do there?
Human: What is 1+1?
AI: 1 + 1 equals 2! It's a simple arithmetic problem, but it's always good to double-check the basics. Do you have any other questions or topics you'd like to discuss?
Human: What is my name?
AI:

> Finished chain.


"Your name is Anirban! It's nice to know your name. Is there anything specific you'd like to talk about today?"

In [58]:
from langchain_classic.memory import ConversationBufferWindowMemory

In [59]:
memory = ConversationBufferWindowMemory(k=1)               

In [61]:
conversation = ConversationChain(
    llm=chat,
    memory = memory,
    verbose=True
)

In [62]:
conversation.predict(input="Hi, I am going to Tokyo tomorrow")
conversation.predict(input="My luggage needs to be picked up from my Osaka hotel")
conversation.predict(input="It needs to be delivered to the Akamai office near Tokyo station")
print(conversation.predict(input="Where am I going tomorrow ?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I am going to Tokyo tomorrow
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I am going to Tokyo tomorrow
AI: That sounds exciting! Tokyo is such a vibrant city with so much to see and do. Do you have any specific plans or places you want to visit while you're there? There are amazing neighborhoods like Shibuya and Shinjuku, beautiful temples like Senso-